In [43]:
import os
import keras
import cv2
import h5py
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop, Adam
import numpy as np
import random
from   tensorflow.keras.preprocessing.image import img_to_array, load_img
import os
from glob import glob
from keras.models import Model
from keras.layers import Input, Dense, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.preprocessing import image

In [44]:
print(os.listdir("../input/urecasegfire/Train"))
path = '../input/urecafire/Train'
print(path)

['Non-Fire', 'Fire']
../input/urecafire/Train


In [45]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)


In [46]:
#with tpu_strategy.scope():
model = tf.keras.models.Sequential([

tf.keras.layers.Conv2D(16, (3,3), activation='relu',padding = 'same', input_shape=(240, 240, 3)),
#tf.keras.layers.MaxPooling2D(2, 2),

tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
#tf.keras.layers.MaxPooling2D(2,2),

tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),

tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),

tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),

tf.keras.layers.Flatten(),

tf.keras.layers.Dense(1024, activation='relu'),
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.Dense(32, activation='relu'),

tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
          optimizer=Adam(lr=0.001),
          metrics=['accuracy'])



In [47]:
model.summary()



Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 240, 240, 16)      448       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 238, 238, 32)      4640      
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 236, 236, 64)      18496     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 118, 118, 64)      0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 116, 116, 64)      36928     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 58, 58, 64)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 56, 56, 64)       

In [48]:
# from kaggle_datasets import KaggleDatasets
# #print(KaggleDatasets())
# GCS_PATH = KaggleDatasets().get_gcs_path() # you can list the bucket with "!gsutil ls $GCS_PATH"

In [49]:
# # On Kaggle you can also use KaggleDatasets().get_gcs_path() to obtain the GCS path of a Kaggle dataset
# filenames = tf.io.gfile.glob(GCS_PATH) # list files on GCS
# dataset = tf.data.TFRecordDataset(filenames)
# # dataset = dataset.map(...) # TFRecord decoding here...


In [50]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(rescale=1/255)


train_generator = train_datagen.flow_from_directory(
        '../input/urecasegfire/Train',
        target_size=(240, 240),  
        batch_size=128, 

        class_mode='binary')

Found 7529 images belonging to 2 classes.


In [ ]:
file_path = "./Final.h5"
checkpoint = ModelCheckpoint(file_path, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
reduce_on_plateau = ReduceLROnPlateau(monitor="loss", mode="min", factor=0.1, patience=5, verbose=1)
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=1)
callbacks_list = [ checkpoint, reduce_on_plateau, es]

history = model.fit(
      train_generator,
      steps_per_epoch=58, 
      callbacks=callbacks_list,
      #validation_split=0.2,
      #shuffle = True,
      epochs=100,
      verbose=1)

Train for 58 steps
Epoch 1/100
57/58 [============================>.] - ETA: 0s - loss: 0.5434 - accuracy: 0.7295
Epoch 00001: accuracy improved from -inf to 0.73125, saving model to ./Final.h5
58/58 [==============================] - 37s 629ms/step - loss: 0.5415 - accuracy: 0.7313
Epoch 2/100
57/58 [============================>.] - ETA: 0s - loss: 0.4149 - accuracy: 0.8126
Epoch 00002: accuracy improved from 0.73125 to 0.81246, saving model to ./Final.h5
58/58 [==============================] - 35s 605ms/step - loss: 0.4150 - accuracy: 0.8125
Epoch 3/100
57/58 [============================>.] - ETA: 0s - loss: 0.3242 - accuracy: 0.8647
Epoch 00003: accuracy improved from 0.81246 to 0.86475, saving model to ./Final.h5
58/58 [==============================] - 36s 621ms/step - loss: 0.3249 - accuracy: 0.8647
Epoch 4/100
57/58 [============================>.] - ETA: 0s - loss: 0.2337 - accuracy: 0.9061
Epoch 00004: accuracy improved from 0.86475 to 0.90636, saving model to ./Final.h5
58

In [ ]:
path = '../input/onepictest/1.jpg' 
img = image.load_img(path, target_size=(240, 240))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
plt.imshow(img)
plt.show()
classes = model.predict(images, batch_size=10)
print(classes[0])
if classes[0]>0.5:
    print(  " is a Fire")
else:
    print( " is a No Fire")
    